# Prepare Work

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from pandas import DataFrame, Series

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
free_throw_all = pd.read_csv('../input/free_throws.csv')
free_throw_all.tail()

In [ ]:
free_throw_all.head()

# Describetion of dataset
- **end_result**: host total score - guest total score
- **game**: host team vs guest team
- **game_id**: id of specific game
- **period**: which quater
- **play**: who make free throw, make or miss free throw
- **player**: player name
- **playoffs**: whether a playoff game or regular game
- **score**: host team score - guest team score at that time
- **season**: NBA season
- **shot_made**: whether player got the free throw
- **time**: time left in that quater

## Hit rate since 2006 season
Let us just compute players' total free throw shot, scores made by free throw and hit rate different seasons.

In [ ]:
free_throw_player = free_throw_all[['season', 'player', 'shot_made']]
free_throw_player.head()

In [ ]:
overview = pd.pivot_table(free_throw_player,
               values='shot_made',
               index=['season', 'player'],
               aggfunc=[np.sum, len, np.mean])
overview.columns = ['score', 'total_shot', 'hit rate']

In [ ]:
noise = overview.total_shot < 80
noise_ind = overview[noise].index
overview = overview.drop(noise_ind)

In [ ]:
overview.sort_values('hit rate', ascending=False).iloc[:10]

Let's see the most stabile free throw players' list. Jose Calderon get the first which really suprise me, because I've seen little of his game in 2008-2009 season, he's performance was amazing that year, **just lost 3 balls!?** My gosh. =。= There are several unfamiliar names in the list, such as Brian Roberts, Anthony Morrow and Peja Stojakovic. And all other 4 players are good shooters, if you leave them in 3-pt line uncovered, they will punish you. **So I think probably 3-pt hit rate is strongly relating with free throw hit rate, but this dataset does not include 3-pt relating data.**

## Compare player free throw performance in decisive moment
I define decisive moment to be **last minutes in a tight game(score gap smaller than 10 points).** Because in a tight game free throw performance probably would lead game into different result, if you've ever seen Hack-a-Shaq. Plus, if a team just lead 1 pont, then a miss would let the enemy get chance to tie the game or even overtake the lead.Thus a player's free throw performance is stabile or not can really important.

In [ ]:
from datetime import datetime
ptime = lambda x: datetime.time(datetime.strptime(x, '%M:%S'))

In [ ]:
free_throw_all['time'] = free_throw_all.time.map(ptime)

In [ ]:
free_throw_all.tail()

In [ ]:
def score_to_score(x):
    piece = [int(y.strip()) for y in x.split('-')]
    if abs(piece[0] - piece[1]) < 10:
        return True
    else:
        return False

In [ ]:
free_throw_all['tight_game'] = free_throw_all.end_result.map(score_to_score)
free_throw_all.tail()

In [ ]:
from datetime import time
last_two_minutes = time(hour=0, minute=2, second=0)
last_quater = free_throw_all.period == 4.0
last_2min = free_throw_all.time < last_two_minutes

In [ ]:
free_throw_all.where(last_quater).where(last_2min).where(free_throw_all.tight_game).dropna()

In [ ]:
count1 = lambda x: sum(1 for y in x)
regular = free_throw_all.playoffs == 'regular'
regular_d = free_throw_all[regular]
regular_D = regular_d.pivot_table(values='shot_made',
                      index=['season', 'player'],
                      aggfunc=[np.sum, len, np.mean])
regular_D.columns = ['score', 'totalshot', 'hit_rate']

In [ ]:
playoffs = free_throw_all.playoffs == 'playoffs'
playoffs_d = free_throw_all[playoffs]
playoffs_D = playoffs_d.pivot_table(values='shot_made',
                      index=['season', 'player'],
                      aggfunc=[np.sum, len, np.mean])
playoffs_D.columns = ['score', 'totalshot', 'hit_rate']

In [ ]:
reg_vs_poff = pd.merge(regular_D, playoffs_D, left_index=True, right_index=True, suffixes=('_regular', '_playoffs'))

In [ ]:
noise = reg_vs_poff.totalshot_playoffs < 5
noise_ind = reg_vs_poff[noise].index
reg_vs_poff = reg_vs_poff.drop(noise_ind)

In [ ]:
reg_vs_poff

In [ ]:
reg_vs_poff.sort_values(['totalshot_regular', 'hit_rate_regular'], ascending=False).iloc[:11]

#### Regular game performance
Let's see who is in top10 free throw list. Wow ヾ(=･ω･=)o, poor **Dwight Howard** strongly leads the rank and he gets 4 position in this narrow list. Take his poor hit rate into consideration, it's not suprise why he would get much more chance than other palyer to stand to the line, **nearly every coach wants to narrow gap by force him making free throw.**
![Dwight Howard](http://ojr2ayzzn.bkt.clouddn.com/snipaste_20170713_134428.png)

Except Dwight Howard, all other 6 player is good at crash enemy's defence line and make body contact. Let's see, Kevin Durant, a man with amazing long arm; James Harden, a player especially good at creating foul by fake and then make the shot; Allen Iverson and  Dwyane Wade both are my favoriate players, and both of them are amazing quick and know how to make breakthrough. **More important, all of them have higher hit rate.**

In [ ]:
reg_vs_poff.sort_values(['totalshot_playoffs', 'hit_rate_playoffs'], ascending=False).iloc[:11]

#### Playoffs games performance
During playoffs, NBA games lift into another level, they are more intense. Of course, how many round players fought through and how many playoffs games they played both greatly affect total shot number. This time LeBron James without doubt take the lead. **Consider his strong body controll ability(he always makes incrediable 2+1 shot, monster...), his truly making fouls ability might be understated.** 2011-2012 season, he was in Heat and that year he got Championship, combining with his ‘Tough guy’ style basketball, it's not suprise that he got so many chance to make free throws.

And let's see another great player, Kobe Bryant. 2008 - 2009 and 2007 - 2008 he twice crashed into NBA finals against same counter, Boston Celtics. The former year, he lost in Boston, and as we can see, his hit rate is not very high, just beyond 80%. The latter year, he's performance improved a lot, nearly same total free throw number, but just lost 22 throws. **I know free throw hit rate does not strict related with champion, but in such a fierce series, a little effort might make a different.**
![](http://ojr2ayzzn.bkt.clouddn.com/snipaste_20170713_144340.png)

In [ ]:
reg_vs_poff[['hit_rate_playoffs']].plot(kind='hist')

In [ ]:
AllStar_2016 = DataFrame({'player': ['Dwyane Wade', 'Kyle Lowry',
                          'LeBron James', 'Paul George', 'Carmelo Anthony',
                          'Stephen Curry', 'Russell Westbrook', 'Kobe Bryant',
                          'Kevin Durant', 'Kawhi Leonard'],
                          'year': [2003, 2006, 2003, 2010, 2003, 2009, 2008,
                          1996, 2007, 2011],
                          'position': ['G', 'G', 'F', 'F', 'F', 'G', 'G',
                                       'G', 'F', 'F']})

In [ ]:
def AllStar(x):
    b = x in AllStar_2016['player'].values
    return b

list = free_throw_all['player'].map(AllStar)
AllStar_16 = free_throw_all[list]

In [ ]:
AllStar_2016 = pd.merge(AllStar_16, AllStar_2016,
                        left_on='player', right_on='player',
                        how='inner')
AllStar_2016.tail()

In [ ]:
Year = lambda x: int(x[-4:])
AllStar_2016['Year'] = AllStar_2016['season'].map(Year)

In [ ]:
AllStar_2016['career'] = AllStar_2016['Year'] - AllStar_2016['year']

In [ ]:
table = AllStar_2016.pivot_table(index=['player', 'Year'],
                         values='shot_made',
                         aggfunc=np.mean)

In [ ]:
table = table.unstack()

In [ ]:
table = table.T

In [ ]:
fig, axes = plt.subplots(5, 2, sharey=True, sharex=True)
index = 0
for i in range(5):
    for j in range(2):
        table.iloc[:, index].plot(ax=axes[i, j],
                            figsize=(20, 20),
                            ylim=[0.6, 1],
                            legend=False,
                            yticks=[0.6, 0.8, 1],
                            title=str(table.columns[index]))
        index += 1

# Analysis
After looked graphs, I found **Kevin Durant is the most stable player and he always keep high free throw rate**. And the who perform worst is **LeBron James**! Beside Anthony, he is the only one who could play as C when in the game, we have to know when he was in Miami Heat, he often play as PG or C, and I guess those strong body contacts influences his shooting. 

Most of them have average 80% free shoot rate, even Stephen Curry have bad performance sometimes. But another AllStar player raise my attention, he is **Kawhi Leonard**. As you can see, since he enter NBA, his free shooting rate keeps rase up, if you know a little about him, you may notice he's progress in other fields, such as 3 points, mid range and organize team. 

In [ ]:
free_throw_all.pivot_table(index='period', values='shot_made', aggfunc=np.mean)